<a href="https://colab.research.google.com/github/manyakhare86/Twitter-Hate-Speech-Detection/blob/main/hate_speech_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Reading dataset

In [ ]:
dataset = pd.read_csv('labeled_data.csv')

In [ ]:
dataset.head(10)

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
5,5,3,1,2,0,1,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just..."
6,6,3,0,3,0,1,"!!!!!!""@__BrighterDays: I can not just sit up ..."
7,7,3,0,3,0,1,!!!!&#8220;@selfiequeenbri: cause I'm tired of...
8,8,3,0,3,0,1,""" &amp; you might not get ya bitch back &amp; ..."
9,9,3,1,2,0,1,""" @rhythmixx_ :hobbies include: fighting Maria..."


In [ ]:
dataset.drop(dataset.columns[[0]], axis=1, inplace=True)

# Swapinging columns

In [ ]:
swap_col = ["count","offensive_language","neither","class","tweet","hate_speech"]
dataset = dataset.reindex(columns=swap_col)
dataset['tweet']


0        !!! RT @mayasolovely: As a woman you shouldn't...
1        !!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2        !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3        !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4        !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
                               ...                        
24778    you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779    you've gone and broke the wrong heart baby, an...
24780    young buck wanna eat!!.. dat nigguh like I ain...
24781                youu got wild bitches tellin you lies
24782    ~~Ruffled | Ntac Eileen Dahlia - Beautiful col...
Name: tweet, Length: 24783, dtype: object

# Downloading nltk

In [ ]:
import nltk
nltk.download('all-nltk')
print("\n")

[nltk_data] Downloading collection 'all-nltk'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    

[nltk_data]    |   Unzipping corpora/ycoe.zip.
[nltk_data]    | 
[nltk_data]  Done downloading collection all-nltk


In [ ]:
from nltk.tokenize import word_tokenize

sentences = []
for i in dataset['tweet']:
    sentences.append(i)
sentences

["!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...",
 '!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!',
 '!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit',
 '!!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny',
 '!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;',
 '!!!!!!!!!!!!!!!!!!"@T_Madison_x: The shit just blows me..claim you so faithful and down for somebody but still fucking with hoes! &#128514;&#128514;&#128514;"',
 '!!!!!!"@__BrighterDays: I can not just sit up and HATE on another bitch .. I got too much shit going on!"',
 "!!!!&#8220;@selfiequeenbri: cause I'm tired of you big bitches coming for us skinny girls!!&#8221;",
 '" &amp; you might not get ya bitch back &amp; thats t

# Text preprocessing



In [ ]:
dataset['tweet'].head(1)

0    !!! RT @mayasolovely: As a woman you shouldn't...
Name: tweet, dtype: object

In [ ]:
# \w = [a-zA-Z0-9_] 
# (?:...) = A non-capturing version of regular parentheses. Matches whatever regular expression is inside the parentheses, but the substring matched 
# by the group cannot be retrieved after performing a match or referenced later in the pattern.

import re

def remove_prior_text(single_tweet):
    remove_user_name = re.sub(r'(@\w+)',"",single_tweet)
    remove_url = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',"",remove_user_name)
    remove_symbols = re.sub(r'([^A-Za-z0-9 ]+)',"",remove_url)
    remove_RT = re.sub(r'RT',"",remove_symbols)
    remove_num = re.sub(r'[0-9]{1,}',"", remove_RT)
    
   
    return remove_num


In [ ]:
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

all_sentences_with_preprocessing = []

for sentence in sentences:
    
    regex_filtered_sen = remove_prior_text(sentence)
    without_punc_lst = []
    for word in regex_filtered_sen:
        # removing punctuations
        if word not in string.punctuation:

            # lowering word
            lower_word = word.lower()
            without_punc_lst.append(lower_word)

    # tokenization 
    sen_lst = word_tokenize("".join(without_punc_lst))

    # removing stopwords
    stop_words = set(stopwords.words('english'))
    filtered_sen = [single_word for single_word in sen_lst if single_word not in stop_words]
    
    # lemmatization
    lemmatized_lst = [lemmatizer.lemmatize(word, pos='n') for word in filtered_sen]
    lemmatized_lst
    all_sentences_with_preprocessing.append(' '.join(lemmatized_lst))
len(all_sentences_with_preprocessing)              

24783

In [ ]:
all_sentences_with_preprocessing_df = pd.DataFrame(all_sentences_with_preprocessing)

In [ ]:
all_sentences_with_preprocessing_df

,0
0,woman shouldnt complain cleaning house amp man...
1,boy dat coldtyga dwn bad cuffin dat hoe st place
2,dawg ever fuck bitch start cry confused shit
3,look like tranny
4,shit hear might true might faker bitch told ya
...,...
24778,yous muthafin lie right tl trash mine bible sc...
24779,youve gone broke wrong heart baby drove rednec...
24780,young buck wan na eat dat nigguh like aint fuc...
24781,youu got wild bitch tellin lie


In [ ]:
dataset['tweet'] = all_sentences_with_preprocessing_df

In [ ]:
dataset.head(10)

,count,offensive_language,neither,class,tweet,hate_speech
0,3,0,3,2,woman shouldnt complain cleaning house amp man...,0
1,3,3,0,1,boy dat coldtyga dwn bad cuffin dat hoe st place,0
2,3,3,0,1,dawg ever fuck bitch start cry confused shit,0
3,3,2,1,1,look like tranny,0
4,6,6,0,1,shit hear might true might faker bitch told ya,0
5,3,2,0,1,shit blow meclaim faithful somebody still fuck...,1
6,3,3,0,1,sit hate another bitch got much shit going,0
7,3,3,0,1,cause im tired big bitch coming u skinny girl,0
8,3,3,0,1,amp might get ya bitch back amp thats,0
9,3,2,0,1,hobby include fighting mariambitch,1


In [ ]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [ ]:
X

array([[3, 0, 3, 2,
        'woman shouldnt complain cleaning house amp man always take trash'],
       [3, 3, 0, 1, 'boy dat coldtyga dwn bad cuffin dat hoe st place'],
       [3, 3, 0, 1, 'dawg ever fuck bitch start cry confused shit'],
       ...,
       [3, 3, 0, 1,
        'young buck wan na eat dat nigguh like aint fuckin dis'],
       [6, 6, 0, 1, 'youu got wild bitch tellin lie'],
       [3, 0, 3, 2,
        'ruffled ntac eileen dahlia beautiful color combination pink orange yellow amp white coll']],
      dtype=object)

In [ ]:
y

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
words_set = set()

In [ ]:
for sentence in X[:,-1]:
    words_set = words_set.union(set(sentence.split()))
len(words_set)                   

20204

In [ ]:
words_set

{'ridiculous',
 'esj',
 'waitkarma',
 'fall',
 'stealing',
 'thatyou',
 'resurrection',
 'thatsaproblem',
 'hookup',
 'quiet',
 'till',
 'west',
 'poked',
 'twisted',
 'callups',
 'roasted',
 'stressed',
 'winnnnnnnnn',
 'ltlt',
 'overme',
 'password',
 'phifes',
 'wokeup',
 'asa',
 'pregnant',
 'unify',
 'bitchget',
 'clicquot',
 'writes',
 'christianity',
 'agreement',
 'wapos',
 'craig',
 'attractive',
 'yeaa',
 'headless',
 'complaining',
 'chubbies',
 'bomb',
 'omm',
 'curledfrost',
 'insane',
 'butta',
 'tumbled',
 'hehe',
 'wilt',
 'trifling',
 'beautiful',
 'entertaining',
 'clothesthrow',
 'rotthrow',
 'brenda',
 'salivating',
 'closet',
 'diff',
 'borrow',
 'comeup',
 'fg',
 'wheelin',
 'labelling',
 'girlfriend',
 'wuss',
 'sza',
 'drawing',
 'jig',
 'signedtothestreets',
 'monetary',
 'slingy',
 'phenomenally',
 'illneverunderstandwhy',
 'segregation',
 'namefaggots',
 'recruited',
 'billy',
 'meerlozewoordenpuntjebijpaaltje',
 'flashwhen',
 'cuba',
 'foxtrotuniformcharliek

In [ ]:
len_lst = []
for sentence in X[:,-1]:
    len_lst.append(len(sentence.split()))
max(len_lst) 

25

In [ ]:
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
samples = all_sentences_with_preprocessing
samples

['woman shouldnt complain cleaning house amp man always take trash',
 'boy dat coldtyga dwn bad cuffin dat hoe st place',
 'dawg ever fuck bitch start cry confused shit',
 'look like tranny',
 'shit hear might true might faker bitch told ya',
 'shit blow meclaim faithful somebody still fucking hoe',
 'sit hate another bitch got much shit going',
 'cause im tired big bitch coming u skinny girl',
 'amp might get ya bitch back amp thats',
 'hobby include fighting mariambitch',
 'keeks bitch curve everyone lol walked conversation like smh',
 'murda gang bitch gang land',
 'hoe smoke loser yea go ig',
 'bad bitch thing like',
 'bitch get',
 'bitch nigga miss',
 'bitch plz whatever',
 'bitch love',
 'bitch get cut everyday b',
 'black bottle amp bad bitch',
 'broke bitch cant tell nothing',
 'cancel bitch like nino',
 'cant see hoe wont change',
 'fuck bitch dont even suck dick kermit video bout fuck ig',
 'got ya bitch tip toeing hardwood floor',
 'pussy lip like heaven door',
 'hoe hitting

In [ ]:
validation_split = 0.2
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
val_samples = samples[-num_validation_samples:]
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]

NameError: ignored

In [ ]:
from tensorflow.keras.layers import TextVectorization

vectorizer = TextVectorization(max_tokens=17787, output_sequence_length=25)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(25)
vectorizer.adapt(text_ds)

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2023-02-22 11:26:49--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-02-22 11:26:49--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-02-22 11:26:49--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
import os
path_to_glove_file = os.path.join(
    os.path.expanduser("~"), ".keras/datasets/glove.6B.100d.txt"
)

embeddings_index = {}
with open('glove.6B.100d.txt') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [ ]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [ ]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 12229 words (5486 misses)


In [ ]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [ ]:
dataset['hate_speech'].value_counts()[0]

19790

In [ ]:
dataset['hate_speech'].value_counts()[3]

287

In [ ]:
dataset['hate_speech'].value_counts()[1]

3419

In [ ]:
dataset['hate_speech'].value_counts()[2]

1251

In [ ]:
class_names = [0,1,2,3]

In [ ]:
from tensorflow.keras import layers

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(len(class_names), activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         1771700   
                                                                 
 conv1d_3 (Conv1D)           (None, None, 128)         64128     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, None, 128)        0         
 1D)                                                             
                                                                 
 conv1d_4 (Conv1D)           (None, None, 128)         82048     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, None, 128)        0         
 1D)                                                         

In [ ]:
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)

NameError: ignored